In [1]:
import pandas as pd
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import folium
from folium.plugins import FastMarkerCluster

In [2]:
locator = Nominatim(user_agent="myGeocoder")
location = locator.geocode("Champ de Mars, Paris, France")

In [3]:
print(location.address)
print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))

Champ de Mars, Avenue Pierre Loti, Quartier du Gros-Caillou, Paris 7e Arrondissement, Paris, Île-de-France, France métropolitaine, 75007, France
Latitude = 48.85614465, Longitude = 2.297820393322227


In [287]:
df = pd.read_csv("dropoff_locations.csv")
df.head()

,Number,St,Unit,City,State,Delivery_Rt_ID
0,350,3rd St,#1211,Cambridge,MA,1
1,164,5th St,#1,Cambridge,MA,1
2,217,Auburn St,#1,Cambridge,MA,1
3,54,Berkshire St,#1,Cambridge,MA,1
4,243,Broadway,#610,Cambridge,MA,1


In [288]:
df['ADDRESS'] = df['Number'].astype(str) + ' ' + \
                df['St'] + ',' + \
                df['City'] + ',' + \
                df['State'] + ',' + ' USA'   

df.head()

,Number,St,Unit,City,State,Delivery_Rt_ID,ADDRESS
0,350,3rd St,#1211,Cambridge,MA,1,"350 3rd St,Cambridge,MA, USA"
1,164,5th St,#1,Cambridge,MA,1,"164 5th St,Cambridge,MA, USA"
2,217,Auburn St,#1,Cambridge,MA,1,"217 Auburn St,Cambridge,MA, USA"
3,54,Berkshire St,#1,Cambridge,MA,1,"54 Berkshire St,Cambridge,MA, USA"
4,243,Broadway,#610,Cambridge,MA,1,"243 Broadway,Cambridge,MA, USA"


In [163]:
from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(locator.geocode, min_delay_seconds=1.3)
df['location'] = df['ADDRESS'].apply(geocode)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('1221 Cambridge St,Cambridge,MA, USA',), **{}).
Traceback (most recent call last):
  File "/Users/ericenglin/opt/anaconda3/lib/python3.7/urllib/request.py", line 1317, in do_open
    encode_chunked=req.has_header('Transfer-encoding'))
  File "/Users/ericenglin/opt/anaconda3/lib/python3.7/http/client.py", line 1244, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/Users/ericenglin/opt/anaconda3/lib/python3.7/http/client.py", line 1290, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/Users/ericenglin/opt/anaconda3/lib/python3.7/http/client.py", line 1239, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/Users/ericenglin/opt/anaconda3/lib/python3.7/http/client.py", line 1026, in _send_output
    self.send(msg)
  File "/Users/ericenglin/opt/anaconda3/lib/python3.7/http/client.py", line 966, in send
    self.conne

In [164]:
# split point column into latitude, longitude and altitude columns
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)
df['point_location'] = df["longitude"].astype(str) + "," + df["latitude"].astype(str)
df.head()

,Number,St,Unit,City,State,Delivery_Rt_ID,ADDRESS,location,point,latitude,longitude,altitude,point_location
0,350,3rd St,#1211,Cambridge,MA,1,"350 3rd St,Cambridge,MA, USA","(350, Third Street, East Cambridge, Cambridge,...","(42.363930822916664, -71.0835468125, 0.0)",42.363931,-71.083547,0.0,"-71.0835468125,42.363930822916664"
1,164,5th St,#1,Cambridge,MA,1,"164 5th St,Cambridge,MA, USA","(162;164, Fifth Street, East Cambridge, Cambri...","(42.36820405, -71.08375762804951, 0.0)",42.368204,-71.083758,0.0,"-71.08375762804951,42.36820405"
2,217,Auburn St,#1,Cambridge,MA,1,"217 Auburn St,Cambridge,MA, USA","(217, Auburn Street, Cambridgeport, Cambridge,...","(42.36471025, -71.10675534090909, 0.0)",42.364710,-71.106755,0.0,"-71.10675534090909,42.36471025"
3,54,Berkshire St,#1,Cambridge,MA,1,"54 Berkshire St,Cambridge,MA, USA","(54;56, Berkshire Street, East Cambridge, Camb...","(42.3696425, -71.09104734310242, 0.0)",42.369642,-71.091047,0.0,"-71.09104734310242,42.3696425"
4,243,Broadway,#610,Cambridge,MA,1,"243 Broadway,Cambridge,MA, USA","(243, Broadway, Inman Square, Cambridge, Middl...","(42.367239, -71.09552, 0.0)",42.367239,-71.095520,0.0,"-71.09552,42.367239"


In [165]:
df.loc[df['point'].isnull()]

,Number,St,Unit,City,State,Delivery_Rt_ID,ADDRESS,location,point,latitude,longitude,altitude,point_location
66,1000,Jackon Pl,#37,Cambridge,MA,7,"1000 Jackon Pl,Cambridge,MA, USA",None,None,NaN,NaN,NaN,"nan,nan"
67,117,Jackon Pl,NaN,Cambridge,MA,7,"117 Jackon Pl,Cambridge,MA, USA",None,None,NaN,NaN,NaN,"nan,nan"


In [166]:
df.to_csv("dropoff_locations_clean.csv")

In [167]:
df_new = df.dropna(subset=['point'], how='all')
df_new.latitude.isnull().sum()

0

In [168]:
len(df_new), len(df)

(343, 345)

In [169]:
map1 = folium.Map(
    location=[42.362750, -71.101182],
    tiles='cartodbpositron',
    zoom_start=12,
)

In [170]:
df_new.apply(lambda row:folium.Marker(
    location=[row["latitude"], row["longitude"]]).add_to(map1), axis=1)
map1

In [227]:
locations = df_new[['latitude', 'longitude']]
locationlist = locations.values.tolist()

In [250]:
import hdbscan
rads = np.radians(locationlist)
clusterer = hdbscan.HDBSCAN(min_cluster_size=10, min_samples=1, metric='haversine')
cluster_labels = clusterer.fit_predict(locationlist)
df_new['hbdscan_cluster']=cluster_labels

np.unique(cluster_labels)

array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [291]:
df_new.hbdscan_cluster.value_counts()

 8     38
 1     36
 5     29
 4     29
 9     28
-1     28
 6     27
 10    24
 12    23
 11    23
 2     17
 3     16
 7     15
 0     10
Name: hbdscan_cluster, dtype: int64

In [251]:
len(np.unique(cluster_labels))

14

In [252]:
color_dict = {-1:'darkgreen',0:'darkpurple',1:'green',2:'purple',3:'red',4:'gray',5:'lightblue',
             6:'beige',7:'cadetblue',8:'black',9:'pink',10:'darkred',11:'lightred',
             12:'darkblue',13:'lightgreen',13:'white'}

In [253]:
df_new['color']=df_new['hbdscan_cluster'].map(color_dict)

In [275]:
type(str(df_new['hbdscan_cluster'][1]))

str

In [282]:
map = folium.Map(location=[42.379750, -71.101182], zoom_start=13)
for point in range(0, len(locationlist)):
    folium.Marker(locationlist[point], popup=('Driver Group: '+str(df_new['hbdscan_cluster'][point])),
                 icon=folium.Icon(color=df_new["color"][point])).add_to(map)
    
map.save('DeliveryMaps.html')
map

In [321]:
df_final = df.merge(df_new, on=['Number', 'St', 'Unit', 'City', 'State', 'Delivery_Rt_ID','ADDRESS'],
                    how='left')

In [322]:
df_final['Driver_ID']=df_final['hbdscan_cluster']
df_final = df_final.drop(columns = ['index','altitude',
       'point_location', 'hbdscan_clustere', 'coords', 'location','point','geometry',
       'hbdscan_cluster', 'color'])

df_final.loc[df_final['Driver_ID'].isnull(),'Driver_ID']=99 #if no lat/long found, make driver id 99
df_final.Driver_ID = df_final.Driver_ID.astype(int)

df_final.head()

,Number,St,Unit,City,State,Delivery_Rt_ID,ADDRESS,latitude,longitude,Driver_ID
0,350,3rd St,#1211,Cambridge,MA,1,"350 3rd St,Cambridge,MA, USA",42.363931,-71.083547,6
1,164,5th St,#1,Cambridge,MA,1,"164 5th St,Cambridge,MA, USA",42.368204,-71.083758,9
2,217,Auburn St,#1,Cambridge,MA,1,"217 Auburn St,Cambridge,MA, USA",42.364710,-71.106755,11
3,54,Berkshire St,#1,Cambridge,MA,1,"54 Berkshire St,Cambridge,MA, USA",42.369642,-71.091047,9
4,243,Broadway,#610,Cambridge,MA,1,"243 Broadway,Cambridge,MA, USA",42.367239,-71.095520,-1


In [323]:
df_final.to_excel("delivery_table.xlsx",index=False)